# Trainium2 Stock Estimates

Two approaches to modeling the total stock of Trainium2 chips.

In [ ]:
import squigglepy as sq
import numpy as np
from squigglepy.numbers import K, M

sq.set_seed(42)
np.random.seed(42)
N_SAMPLES = 10_000

## Model 1: Data Center Approach

Estimates total Trainium2 stock based on known data center deployments (Project Rainier).

| Parameter | Central estimate | 90% CI | Notes |
|-----------|------------------|--------|-------|
| New Carlisle Trainium2 | ~714k | 500k to 1M | Amazon confirmed "nearly" 500k by October, 1M planned by EOY |
| Madison Trainium2 | ~320k | ~190k to ~540k | 80% CI of 1.5x converted to 90% CI |
| NC + Madison share of Anthropic's Trn2 | - | 70% to 85% | SemiAnalysis: 15% predates Project Rainier |
| Anthropic's share of all Trainium2 | - | 60% to 80% | Project Rainier is majority of cloud Trainium |

In [ ]:
# Model 1: Data Center Approach

# New Carlisle Trainium2 units (tighter interval due to Amazon statements)
new_carlisle = sq.to(500*K, 1*M)  # 90% CI: 500k to 1M

# Madison Trainium2 units
# 80% CI of 1.5x -> 90% CI multiplier = 1.5^(1.645/1.28) = 1.68x
madison = sq.to(190*K, 540*K)  # 90% CI: ~190k to ~540k

# New Carlisle + Madison share of Anthropic's Trainium2
nc_madison_share = sq.to(0.70, 0.85)  # 90% CI: 70% to 85%

# Anthropic's share of all Trainium2
anthropic_share = sq.to(0.60, 0.80)  # 90% CI: 60% to 80%

# Run simulation
model1_samples = []
for _ in range(N_SAMPLES):
    nc = new_carlisle @ 1
    mad = madison @ 1
    nc_mad_share = nc_madison_share @ 1
    anth_share = anthropic_share @ 1
    
    # Total = (NC + Madison) / (NC+Madison share of Anthropic) / (Anthropic share of total)
    anthropic_total = (nc + mad) / nc_mad_share
    total_trainium2 = anthropic_total / anth_share
    model1_samples.append(total_trainium2)

model1_samples = np.array(model1_samples)

print("Model 1: Data Center Approach")
print("="*50)
print(f"Median: {np.median(model1_samples)/M:.2f}M Trainium2 units")
print(f"90% CI: {np.percentile(model1_samples, 5)/M:.2f}M to {np.percentile(model1_samples, 95)/M:.2f}M")
print(f"Mean:   {np.mean(model1_samples)/M:.2f}M Trainium2 units")

## Model 2: Production Forecast Approach

Estimates total Trainium2 stock using Morgan Stanley's February 2025 production forecast.

| Parameter | Central estimate | 90% CI | Notes |
|-----------|------------------|--------|-------|
| 2025 Trainium2 production | 1.4M | ~1M to ~2M | Morgan Stanley forecast, assumes Trn3 dies reallocated |
| 2024 production relative to 2025 | 20% | 15% to 30% | Based on SemiAnalysis research |

In [ ]:
# Model 2: Production Forecast Approach

# 2025 Trainium2 production
production_2025 = sq.to(1*M, 2*M)  # 90% CI: ~1M to ~2M

# 2024 production as fraction of 2025 production
prod_2024_relative = sq.to(0.15, 0.30)  # 90% CI: 15% to 30%

# Run simulation
model2_samples = []
for _ in range(N_SAMPLES):
    p2025 = production_2025 @ 1
    p2024_rel = prod_2024_relative @ 1
    
    # Total = 2025 production + (2024 production = 2025 * relative fraction)
    p2024 = p2025 * p2024_rel
    total_trainium2 = p2025 + p2024
    model2_samples.append(total_trainium2)

model2_samples = np.array(model2_samples)

print("Model 2: Production Forecast Approach")
print("="*50)
print(f"Median: {np.median(model2_samples)/M:.2f}M Trainium2 units")
print(f"90% CI: {np.percentile(model2_samples, 5)/M:.2f}M to {np.percentile(model2_samples, 95)/M:.2f}M")
print(f"Mean:   {np.mean(model2_samples)/M:.2f}M Trainium2 units")

## Comparison of Models

In [ ]:
print("Comparison of Trainium2 Stock Estimates")
print("="*60)
print(f"{'Model':<35} {'Median':>10} {'90% CI':>20}")
print("-"*60)
print(f"{'Model 1: Data Center Approach':<35} {np.median(model1_samples)/M:>10.2f}M {f'{np.percentile(model1_samples, 5)/M:.1f}M - {np.percentile(model1_samples, 95)/M:.1f}M':>20}")
print(f"{'Model 2: Production Forecast':<35} {np.median(model2_samples)/M:>10.2f}M {f'{np.percentile(model2_samples, 5)/M:.1f}M - {np.percentile(model2_samples, 95)/M:.1f}M':>20}")
print("="*60)
print("\nBoth models suggest ~2M is a plausible central estimate for cumulative Trainium2 stock.")
print("Project Rainier (New Carlisle + Madison) makes up roughly half of total deployed Trainium2.")